In [1]:
%pip install pygame -q

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install torch torchvision torchaudio -q --index-url https://download.pytorch.org/whl/cu121


Note: you may need to restart the kernel to use updated packages.


In [1]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "None")


CUDA available: True
GPU name: NVIDIA GeForce GTX 1070


In [ ]:
%pip install deepface

In [ ]:
%pip install tf_keras -q

In [11]:
import cv2

cap = cv2.VideoCapture(3)
if not cap.isOpened():
    print("Cannot open camera")
else:
    ret, frame = cap.read()
    if not ret:
        print("Can't receive frame (stream end?).")
    else:
        cv2.imshow("Webcam test", frame)
        cv2.waitKey(0)
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
%pip install mediapipe -q

In [1]:
import pygame
import cv2
from deepface import DeepFace
import numpy as np

# Setup PyGame window
pygame.init()
WIDTH, HEIGHT = 640, 480
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Real-Time Age/Gender/Emotion")
font = pygame.font.SysFont("Segoe UI", 12)

# Setup webcam
cap = cv2.VideoCapture(3)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, HEIGHT)

# Frame skipping for better performance
frame_skip = 3
frame_count = 0
analysis_result = []

clock = pygame.time.Clock()
running = True

DETECTOR_MODEL = "ssd"  # fast, perhaps not as accurate
# DETECTOR_MODEL = "retinaface"  # very slow, but accurate
# DETECTOR_MODEL = "mtcnn"  # fairly fast and accurate
# DETECTOR_MODEL = "opencv"  # fast, but not as accurate
# DETECTOR_MODEL = "mediapipe"  # very fast, but not as accurate

while running:
    clock.tick(30)  # Target ~30 FPS
    frame_count += 1

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    ret, frame = cap.read()
    if not ret:
        continue

    frame = cv2.resize(frame, (WIDTH, HEIGHT))

    # Run DeepFace analysis every few frames
    if frame_count % frame_skip == 0:
        try:
            analysis_result = DeepFace.analyze(
                frame,
                actions=['age', 'gender', 'emotion'],
                enforce_detection=False,
                detector_backend=DETECTOR_MODEL,
                silent=True
            )
        except Exception as e:
            print("Analysis error:", e)
            analysis_result = []

    # Display video in PyGame
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    surface = pygame.surfarray.make_surface(np.flip(np.rot90(rgb_frame), axis=0))
    screen.blit(surface, (0, 0))

    # Draw results
    if analysis_result:
        for face in analysis_result:
            region = face['region']
            x, y, w, h = region['x'], region['y'], region['w'], region['h']

            age = int(face['age'])
            emotion = face['dominant_emotion']
            gender_probs = face['gender']
            gender = (
                f"Female ({gender_probs['Woman']:.0f}%)"
                if gender_probs['Woman'] > gender_probs['Man']
                else f"Male ({gender_probs['Man']:.0f}%)"
            )
            label = f"{gender} | {age} y/o | {emotion}"

            pygame.draw.rect(screen, (0, 255, 0), pygame.Rect(x, y, w, h), 2)

            # Draw readable label
            text_surface = font.render(label, True, (255, 255, 255))
            text_rect = text_surface.get_rect()
            text_rect.centerx = x + w // 2
            text_rect.top = y - text_rect.height - 2  # a little gap above the box


            bg_surface = pygame.Surface((text_rect.width, text_rect.height))
            bg_surface.set_alpha(150)
            bg_surface.fill((0, 0, 0))
            screen.blit(bg_surface, text_rect)
            screen.blit(text_surface, text_rect)

    pygame.display.update()

cap.release()
pygame.quit()


E:\Data\PyCharmProjects\DeepFace\.venv\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


pygame 2.6.1 (SDL 2.28.4, Python 3.12.10)
Hello from the pygame community. https://www.pygame.org/contribute.html

